# JAI 2022 - Curso de PLN

---

## Seção 1.3. Preparação de Dados Textuais

# Instalação de pacotes adicionais

In [1]:
%%capture
!pip install unidecode
!python -m spacy download pt_core_news_sm

# Importações Necessárias

In [ ]:
import string
from unidecode import unidecode

import numpy as np

# Uso da biblioteca NLTK para estemização
from nltk.stem.snowball import PortugueseStemmer

# Uso da biblioteca spaCy para lematização
import spacy

# Padronização

A função ```standardize()``` irá

*   Transformar todas as letras em letras minúsculas com a função ```lower()```
*   Remover todos os acentos e pontuações presentes em ```string.punctuation```



In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def standardize(text):
    text = unidecode(text.lower())
    return "".join(char for char in text
        if char not in string.punctuation)

In [ ]:
standardize("O gato caçou o rato!")

'o gato cacou o rato'

# Tokenização

Uma tokenização simples consiste na criação de tokens a nível de palavras. Ou seja, com a função ```tokenize()``` vamos criar uma lista de palavras a partir de um texto padronizado com a função ```standardize()```.

In [ ]:
def tokenize(text):
    text = standardize(text) 
    return text.split()

In [ ]:
tokenize("O gato caçou o rato!")

['o', 'gato', 'cacou', 'o', 'rato']

## Estemização

A estemização vai extrair o radical das palavras. Para isso, utilizaremos o ```PortugueseStemmer()```.

In [ ]:
stemmer = PortugueseStemmer()

In [ ]:
print(stemmer.stem("Presidente"))
print(stemmer.stem("Presidenta"))

president
president


In [ ]:
print(stemmer.stem("estava"))
print(stemmer.stem("estive"))

estav
estiv


In [ ]:
print(stemmer.stem("casa"))
print(stemmer.stem("casas"))

cas
cas


In [ ]:
def stem(text):
  return [stemmer.stem(w) for w in tokenize(text)]

In [ ]:
stem("Os gatos caçaram os ratos!")

['os', 'gat', 'cac', 'os', 'rat']

In [ ]:
stem("O gato caçou o rato!")

['o', 'gat', 'cac', 'o', 'rat']

## Lematização

Faremos uma análise mais morfológica das palavras com o processo de lematização. Para isso, utilizaremos a biblioteca Spacy.

In [ ]:
nlp = spacy.load('pt_core_news_sm')

In [ ]:
def lemma(text):
  doc = nlp(standardize(text))
  return [token.lemma_ for token in doc]

In [ ]:
lemma("Amigos, amizade e carreira são importantes")

['amigo', 'amizade', 'e', 'carreira', 'sao', 'importante']

In [ ]:
lemma("Amigas, amizades e carreira é importante")

['amiga', 'amizade', 'e', 'carreira', 'e', 'importante']

# Indexação do Vocabulário

Precisamos transformar as palavras pré-processadas dos nossos textos de entrada em suas representações numéricas.

Faremos isso criando um vocabulário das palavras e, em seguida, representando esse vocabulário em formato vetorial com uma codificação *one-hot*.

## Criação do Vocabulário

In [ ]:
def make_vocab(texts): 
    vocab = {"": 0, "[UNK]": 1} 
    
    for text in texts:
      text = standardize(text) 
      tokens = tokenize(text) 
      
      for token in tokens:
        if token not in vocab: 
          vocab[token] = len(vocab)

    inverse_vocab = dict(
      (v, k) for k, v in vocab.items())
      
    return vocab, inverse_vocab

In [ ]:
vocab, _ = make_vocab(["O gato comeu o rato", "Gato e cachorro são animais"])

vocab

{'': 0,
 '[UNK]': 1,
 'animais': 9,
 'cachorro': 7,
 'comeu': 4,
 'e': 6,
 'gato': 3,
 'o': 2,
 'rato': 5,
 'sao': 8}

## Codificação *One-Hot*

In [ ]:
def one_hot_encode(text, vocab):
    tokens = tokenize(text)
    vectors = np.zeros((len(tokens), len(vocab)))
    
    for i, token in enumerate(tokens):
      token_idx = vocab.get(token, 1)
      vectors[i, token_idx] = 1  
    
    return vectors

In [ ]:
one_hot_encode("O gato comeu o rato", vocab)

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])